In [1]:
import os
import pandas as pd

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models # 이미지
from torchvision import transforms
from PIL import Image

# from transformers import GPT2Tokenizer, GPT2Model # 텍스트
from transformers import BartTokenizer, BartModel # 텍스트

from tqdm.auto import tqdm

C:\Users\user\anaconda3\envs\DACON\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class VQADataset(Dataset):
    def __init__(self, df, tokenizer, transform, img_path, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.transform = transform
        self.img_path = img_path
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg') # 이미지
        image = Image.open(img_name).convert('RGB')
        image = self.transform(image)

        question = row['question'] # 질문
        question = self.tokenizer.encode_plus(
            question,
            truncation=True,
            add_special_tokens=True,
            max_length=32,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        if not self.is_test:
            answer = row['answer'] # 답변
            answer = self.tokenizer.encode_plus(
                answer,
                max_length=32,
                padding='max_length',
                truncation=True,
                return_tensors='pt')
            return {
                'image': image.squeeze(),
                'question': question['input_ids'].squeeze(),
                'answer': answer['input_ids'].squeeze()
            }
        else:
            return {
                'image': image,
                'question': question['input_ids'].squeeze(),
            }

In [3]:
class VQAModel(nn.Module):
    def __init__(self, vocab_size):
        super(VQAModel, self).__init__()
        self.vocab_size = vocab_size

        self.effi = models.efficientnet_v2_l(pretrained=True)
        self.bart = BartModel.from_pretrained('facebook/bart-base')
        self.bart.resize_token_embeddings(vocab_size) # 추가한 [PAD] 토큰 반영

        combined_features_size = 1000 + self.bart.config.hidden_size # resnet 출력 차원 + bart 출력 차원
        self.classifier = nn.Linear(combined_features_size, vocab_size)

    def forward(self, images, question):
        image_features = self.effi(images)
        image_features = image_features.view(image_features.size(0),-1)

        outputs = self.bart(question)
        output_features = outputs.last_hidden_state # [batch, sequence, hidden]

        image_features = image_features.unsqueeze(1).expand(-1, output_features.size(1),-1) # [batch, sequence, 1000]

        combined = torch.cat([image_features, output_features], dim=-1) # [batch, sequence, 1000+hidden]
        output = self.classifier(combined) # [batch, vocab_size]
        return output

In [4]:
# 데이터 불러오기
train_df = pd.read_csv('./open/train.csv')
test_df = pd.read_csv('./open/test.csv')
sample_submission = pd.read_csv('./open/sample_submission.csv')
train_img_path = './open/image/train'
test_img_path = './open/image/test'

# dataset & dataloader
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
vocab_size = len(tokenizer)

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # 수평으로 뒤집기
    transforms.RandomVerticalFlip(),    # 수직으로 뒤집기
    transforms.RandomChoice([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2), # 색 변경
        transforms.RandomAffine(   # 랜덤으로 회전 또는 이동
            degrees=15, translate=(0.2, 0.2),
            scale=(0.8, 1.2), shear=15, resample=Image.BILINEAR)
    ]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = VQADataset(train_df, tokenizer, transform_train, train_img_path, is_test=False)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

C:\Users\user\anaconda3\envs\DACON\lib\site-packages\torchvision\transforms\transforms.py:1427: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed in 0.14. Please use 'interpolation' instead.
  warnings.warn(


In [5]:
def train(model, loader, optimizer, criterion,scheduler):
    model.train()
    total_loss = 0

    for data in tqdm(loader, total=len(loader)):
        images = data['image'].to(device)
        question = data['question'].to(device)
        answer = data['answer'].to(device)

        optimizer.zero_grad()

        outputs = model(images, question)

        # output: [batch, sequence, vocab], answer : [batch, sequence]
        loss = criterion(outputs.view(-1, outputs.size(-1)), answer.view(-1))
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(loader)
    return avg_loss

In [6]:
def inference(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for data in tqdm(loader, total=len(loader)):
            images = data['image'].to(device)
            question = data['question'].to(device)

            outputs = model(images, question) # [batch, sequence, vocab]

            _, pred = torch.max(outputs, dim=2) # values, indices = _, pred
            preds.extend(pred.cpu().numpy())

    return preds

In [ ]:
# device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"current device is {device}")

# Model
model = VQAModel(vocab_size).to(device)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=3)
epochs=6
# Training loop
for epoch in range(epochs):
    avg_loss = train(model, train_loader, optimizer, criterion,scheduler)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")

C:\Users\user\anaconda3\envs\DACON\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\user\anaconda3\envs\DACON\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_L_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


current device is cuda


  0%|          | 73/22471 [00:46<3:33:08,  1.75it/s]

In [ ]:
# Dataset & DataLoader
test_dataset = VQADataset(test_df, tokenizer, transform_test, test_img_path, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# inference
preds = inference(model, test_loader)

no_pad_output = []

for pred in preds:
    output = pred[pred != 50257] # [PAD] token 제외
    no_pad_output.append(tokenizer.decode(output).strip()) # 토큰 id -> 토큰

In [ ]:
sample_submission['answer'] = no_pad_output
sample_submission.to_csv('results/submission_bart_base_ep6_bt16_argu.csv', index=False)